### Create a google api key

In [ ]:
from dotenv import load_dotenv

load_dotenv()

## initialize the LLM Model

In [ ]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("google_genai:gemini-2.0-flash")

# response = llm.invoke("Who was the first person to walk on moon?")
# response.content

### Define a state? and what it is?

when you are exchanging a messages, you should have a messages list, all the messages will be appended here

In [ ]:
from typing import TypedDict
from langgraph.graph import StateGraph, START, END

# Define a state
class State(TypedDict):
    messages: list

# Define a chatbot
def chatbot(state: State) -> State:
    return {"messages": [llm.invoke(state['messages'])]}


# build a nodes
builder = StateGraph(State)
builder.add_node("chatbot", chatbot)

# add edges
builder.add_edge(START, "chatbot")
builder.add_edge("chatbot", END)

graph = builder.compile()
print(graph)

In [ ]:
messages = [{"role": "user", "content": "Who walked on the moon for the first time? Print only the name."}]
response = graph.invoke({'messages': messages})
print(response['messages'][-1].content)

## Complete messages with Chatbot

But here i am unable to see the Human message (Question) and LLM response


Human : WHo is the pm of india?
AI: Modi
Human : What is his age?

In [2]:
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv

load_dotenv()

llm = init_chat_model("google_genai:gemini-2.0-flash")

# response = llm.invoke("Who was the first person to walk on moon?")
# response.content

# add_messages is inbuilt function in langgraph. It will once you specify the syntax like Annotated[list, add_messages]. 
# It will automatically preserves the original message

# Define a state
class State(TypedDict):
    messages: Annotated[list, add_messages]

# Define a chatbot
def chatbot(state: State) -> State:
    return {"messages": [llm.invoke(state['messages'])]}


# build a nodes
builder = StateGraph(State)
builder.add_node("chatbot", chatbot)

# add edges
builder.add_edge(START, "chatbot")
builder.add_edge("chatbot", END)

graph = builder.compile()
print(graph)

c:\Users\virub\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# messages = [{"role": "user", "content": "Who walked on the moon for the first time? Print only the name."}]
# response = graph.invoke({'messages': messages})
# response

In [ ]:
# messages = [{"role": "user", "content": "Which year"}]
# response = graph.invoke({'messages': messages})
# response

## Build a conversational Chatbot

In [ ]:
state = None

while True:
    in_message = input("You: ")
    
    if in_message.lower() in {"quit", "exit"}:
        break
    if state is None:
        state: State = {"messages": [{"role": "user", "content": in_message}]}
    else:
        state['messages'].append({"role": "user", "content": in_message})
    
    state = graph.invoke(state)
    print(state["messages"])
    print("Bot: ", state['messages'][-1].content)
print("exit")

Bot:  The Prime Minister of India is Narendra Modi.
Bot:  Narendra Modi was born on September 17, 1950. So, as of today, November 19, 2023, he is 73 years old.


In [3]:
messages = [{"role": "user", "content": "What is the Latest price of MSFT Stock?"}]
response = graph.invoke({'messages': messages})
response

{'messages': [HumanMessage(content='What is the Latest price of MSFT Stock?', additional_kwargs={}, response_metadata={}, id='7bcc6769-39ad-4a29-a718-37b39c2f8e55'),
  AIMessage(content="As of October 26, 2023, at 2:18 PM PST, the price of MSFT (Microsoft) stock is $330.26.\n\nPlease note that stock prices fluctuate throughout the day, so this is just a snapshot in time. For the most up-to-date information, it's best to check a reliable source like Google Finance, Yahoo Finance, or your brokerage account.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='lc_run--b2524979-c5b2-4e0f-9df3-6144114bfbfe-0', usage_metadata={'input_tokens': 10, 'output_tokens': 92, 'total_tokens': 102, 'input_token_details': {'cache_read': 0}})]}

If you look at the Response, LLM is not providing the Latest Stock Price rate. It is giving till 2023
So to get the latest stock price we need to call the tools to get the latest stock price